# Computing cell mask

## Exercises 1 - Adjust Brightness and Contrast

In [1]:
# (i) Adjust Brightness and Contrast
# Before compute cell mask, try to increase the brigtness and the contrast of the image. 
# Note: Play with mathematical operator
# operator '+' or '-' for the brigthness
# operator '*' for contrast

ntritc = tritc * 2.82 - 0.17
nfitc = fitc * 5.03 - 0.35
nhoechst = hoechst * 2.99 - 0.15

#(ii) Visualize the result after create a mix image using plt.imshow and plt.show
mix = nfitc+ntritc+nhoechst
plt.figure(figsize=(14,14))
plt.imshow(mix, cmap = 'gray')
plt.show()

NameError: name 'tritc' is not defined

In [ ]:
#(iii) Do you improve the light of the image by playing with the gamma.
# test operator ^ (** in python) to apply gamma correction
mix_2 = (nfitc**2+ntritc**2+nhoechst**2)**0.5

#(iv) Visualize the result
plt.figure(figsize=(14,14))
plt.imshow(mix_2, cmap = 'gray')
plt.show()

## 2d convolution filter

Linear filtering is useful to perform low-pass filtering (to blur images, remove noise...) and high-pass filtering (to detect edges, sharpen images).

## Exercises 2 - Apply a convolution filter

In [ ]:
# (i) Create a cross-shaped structuring element and asign it to a new variable.
# Structuring elements are small binary images that indicate which pixels 
# should be considered as the 'neighborhood' of the central pixel.
# create a matrix with 3 columns and 3 rows as below
# 0 1 0
# 1 2 1
# 0 1 0
# then divide all values by 6 to obtain a float type matrix. Use the function matrix from numpy. 
se = np.matrix([[0,1,0],[1,2,1],[0,1,0]], dtype='float')/6

# (ii) import signal from scipy to get the convolve2d function for execute the 2d convolution filter on the image. 
# Read the documentation to get more details on the parameters of the function.
from scipy import signal

cmask = signal.convolve2d(mix_2, se, boundary="wrap", mode="same") >= 0.12

In [ ]:
#(iii) display the result
plt.imshow(cmask,interpolation='none',cmap='gray')
plt.show()

In [ ]:
# (iii) Create a diamond-shaped structuring element and asign it to a new variable.
# Structuring elements are small binary images that indicate which pixels 
# should be considered as the 'neighborhood' of the central pixel.
# You can use skimage.morphology.diamond to create diamond-shaped structuring element
se = diamond(1)

# (iv) Use morphology operation like closing to improve image
cmask = closing(cmask, se)

plt.imshow(cmask)

# segmenting nuclei using watershed

**Background**

The algorithm identifies and separates objects that stand out of the background (zero). It inverts the image and uses water to fill the resulting valleys (pixels with high intensity in the source image) until another object or background is met. The deepest valleys become indexed first, starting from 1.   
An array of seeds contains a few pixels at the center of each cell labeled by a unique ID number and otherwise surrounded by zeros. The expansion algorithm will start from these central pixels and grow outward until all zeros are overwritten by an ID label. In the case of watershed expansion, one can imagine the seeds as the sources from which water pours into the cells and starts filling them up.   
Here, we will use a distance transform and local maxima for seeding.

## Exercises 3 - Seeded watershed

In [ ]:
# Distance transform on thresholded membranes
# Use the function ndi.distance_transform_edt.
dist_transf = ndi.distance_transform_edt(nmask)

In [ ]:
# Visualize the output and understand what you are seeing.
plt.figure(figsize=(14,14))
plt.imshow(dist_transf,interpolation='none',cmap='gray')
plt.show()

In [ ]:
# Dilate the distance threshold
# Use ndi.filters.maximum_filter to dilate the distance transform.
i = 10
struct = (np.mgrid[:i,:i][0] - np.floor(i/2))**2 + (np.mgrid[:i,:i][1] - np.floor(i/2))**2 <= np.floor(i/2)**2
dist_trans_dil = ndi.filters.maximum_filter(dist_transf, footprint=struct) 
plt.figure(figsize=(7,7))
plt.imshow(dist_trans_dil, interpolation='none', cmap='viridis')
plt.show()

In [ ]:
# Retrieve the local maxima (the 'peaks') in the distance transform
# Use the function peak_local_max from the module skimage.feature.
from skimage.feature import peak_local_max
seeds = peak_local_max(dist_trans_dil, indices=False, min_distance=10)

In [ ]:
# Visualize the output
# you can first plot the original input (or the smoothed) image and
# then plot the seeds on top of it before showing both with 'plt.show'.
plt.figure(figsize=(7,7))
plt.imshow(hoechst_smooth, interpolation='none', cmap='gray')
plt.imshow(np.ma.array(seeds,mask=seeds==0),interpolation='none',cmap='autumn')
plt.show()

In [ ]:
# Label the seeds
# Use connected component labeling to give each cell seed a unique ID number.
seeds_labeled = ndi.label(seeds)[0]

In [ ]:
# Perform watershed
# Use the function watershed from the module skimage.morphology.
# Use the labeled nuclei seeds and the smoothed nuclei image as input.
from skimage.morphology import watershed
ws = watershed(hoechst_smooth,seeds_labeled)

In [ ]:
# Show the result as transparent overlay over the smoothed input image
plt.figure(figsize=(7,7))
plt.imshow(hoechst_smooth, interpolation='none', cmap='gray')
plt.imshow(ws,interpolation='none',cmap='prism',alpha=0.4)
plt.show()

# segmenting cells using Voronoi

Find boundaries between adjacent regions in an image, where seeds have been already identified in the individual regions to be segmented. The method finds the Voronoi region of each seed on a manifold with a metric controlled by local image properties. The method is motivated by the problem of finding the borders of cells in microscopy images, given a labelling of the nuclei in the images.

## Exercises 3 - A modified voronoi algorithm

In [ ]:
# The label function from skimage.measure module finds every connected set of pixels other than the background, 
# and relabels these sets with a unique increasing integer.
from skimage.measure import label
markers_nuc = label(nmask, connectivity=2, return_num=True)

In [ ]:
# We apply a sobel filter from skimage.filters to detect edges from cell body and serve as input 
# for the voronoi algorithm. 
from skimage.filters import sobel

fsobel = np.empty_like(mix_2)
fsobel = sobel(mix_2)

# Visualize the output
plt.imshow(fsobel)

In [ ]:
# This algo is implemented in centrosome package, an open source image processing library.
from centrosome.propagate import propagate
cell_seg,_ = propagate(image=fsobel, labels=markers_nuc[0], mask=cmask, weight=1)

# Visualize the output. Focus on boundaries between labeled 
# regions highlighted thanks to mark_boundaries from skimage.segmentation and draw contour lines 
# for nuclei thanks to contour from matplotlib.pyplot.
from skimage.segmentation import mark_boundaries
plt.imshow(mark_boundaries(composite, cell_seg))
plt.contour(nmask, colors = 'r', linewidths = 0.1)